In [42]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

In [43]:
df = pd.read_csv('../output.csv')

In [44]:
df.head()

,s,o,h,l,c,v,t
0,ok,173.26,173.26,173.26,173.26,0.0,869356800
1,ok,173.25,173.25,173.25,173.25,0.0,869443200
2,ok,173.03,173.03,173.03,173.03,0.0,869529600
3,ok,173.03,173.03,173.03,173.03,0.0,869616000
4,ok,172.66,172.66,172.66,172.66,0.0,869702400


In [45]:
df.shape

(6329, 7)

In [46]:
df.columns

Index(['s', 'o', 'h', 'l', 'c', 'v', 't'], dtype='object')

In [47]:
df= df.rename(columns={'s':'s',"o":'opening_price','h':'high_price','l':'lowest_price','c':'closing_price','v':'volume','t':'timestamp'})
df.head()

,s,opening_price,high_price,lowest_price,closing_price,volume,timestamp
0,ok,173.26,173.26,173.26,173.26,0.0,869356800
1,ok,173.25,173.25,173.25,173.25,0.0,869443200
2,ok,173.03,173.03,173.03,173.03,0.0,869529600
3,ok,173.03,173.03,173.03,173.03,0.0,869616000
4,ok,172.66,172.66,172.66,172.66,0.0,869702400


In [48]:
df.drop(['s'],axis = 1,inplace=True)

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6329 entries, 0 to 6328
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   opening_price  6329 non-null   float64
 1   high_price     6329 non-null   float64
 2   lowest_price   6329 non-null   float64
 3   closing_price  6329 non-null   float64
 4   volume         6329 non-null   float64
 5   timestamp      6329 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 296.8 KB


In [50]:
df.describe()

,opening_price,high_price,lowest_price,closing_price,volume,timestamp
count,6329.000000,6329.000000,6329.000000,6329.000000,6.329000e+03,6.329000e+03
mean,944.901435,949.804121,939.267383,944.162002,1.031748e+09,1.314714e+09
std,778.576378,785.827006,769.801524,777.129855,2.667614e+09,2.546312e+08
min,156.430000,156.430000,156.430000,156.430000,0.000000e+00,8.693568e+08
25%,313.000000,313.000000,313.000000,313.000000,0.000000e+00,1.093392e+09
50%,634.000000,634.000000,634.000000,634.000000,0.000000e+00,1.316045e+09
75%,1391.030000,1402.210000,1382.330000,1391.030000,5.158190e+08,1.534637e+09
max,3208.530000,3227.110000,3178.250000,3198.190000,2.995852e+10,1.752106e+09


In [51]:
df['volume'] = df['volume'].astype('int64')

In [52]:
df.timestamp = pd.to_datetime(df.timestamp, unit='s')#.dt.tz_localize("Asia/Kathmandu")

In [53]:
df.timestamp

0      1997-07-20
1      1997-07-21
2      1997-07-22
3      1997-07-23
4      1997-07-24
          ...    
6324   2025-07-06
6325   2025-07-07
6326   2025-07-08
6327   2025-07-09
6328   2025-07-10
Name: timestamp, Length: 6329, dtype: datetime64[ns]

In [54]:
df = df.set_index('timestamp')

In [55]:
df.tail()

,opening_price,high_price,lowest_price,closing_price,volume
timestamp,,,,,
2025-07-06,2708.24,2722.33,2697.32,2706.36,9500003268
2025-07-07,2715.40,2718.10,2680.45,2684.01,8377327147
2025-07-08,2683.77,2690.13,2662.59,2683.86,6574166814
2025-07-09,2679.37,2707.91,2679.16,2705.90,7451877102
2025-07-10,2709.09,2743.09,2706.89,2731.80,9682488739


In [56]:
df.isnull().sum()

opening_price    0
high_price       0
lowest_price     0
closing_price    0
volume           0
dtype: int64

In [57]:
from sklearn.preprocessing import MinMaxScaler

# Select the column you want to predict (typically 'Close' price)
prices = df['closing_price'].values.reshape(-1, 1)

# Normalize the data (critical for neural networks)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_prices = scaler.fit_transform(prices)

In [58]:
def create_sequences(data, seq_length,pred_length=7):
    xs, ys = [], []
    for i in range(len(data)-seq_length-pred_length):
        x = data[i:(i+seq_length)]
        y = data[i+seq_length:(i+seq_length+pred_length)]
        xs.append(x)        
        ys.append(y)
    return np.array(xs), np.array(ys)

# Choose sequence length (e.g., 60 days)
SEQ_LENGTH = 90
PRED_LENGTH = 7  # Predict next 7 days YOU CAN CHANGE THIS AS PER YOUR NEEDS
# Create sequences
X, y = create_sequences(scaled_prices, SEQ_LENGTH, PRED_LENGTH)

# Split into train and test sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [59]:
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length - 1):
        x = data[i:(i + seq_length)]
        y = data[i + seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

SEQ_LENGTH = 30
X, y = create_sequences(scaled_prices, SEQ_LENGTH)
train_size = int(len(X) * 0.8)

X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [60]:
class StockPredictorLSTM(nn.Module):
    def __init__(self, input_size=1, hidden_size=50, num_layers=1, output_size=7): # OUTPUT_SIZE IS 7 BECAUSE WE ARE PREDICTING NEXT 7 DAYS
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        # Initialize hidden state and cell state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).requires_grad_()
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).requires_grad_()
        
        # Forward propagate LSTM
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out

In [61]:
X_train_tensor = torch.FloatTensor(X_train).reshape(-1, SEQ_LENGTH, 1)
X_test_tensor = torch.FloatTensor(X_test).reshape(-1, SEQ_LENGTH, 1)
y_train_tensor = torch.FloatTensor(y_train)
y_test_tensor = torch.FloatTensor(y_test)

# Ensure targets are (batch_size, 7)
if y_train_tensor.ndim == 3:
    y_train_tensor = y_train_tensor.squeeze(-1)
    y_test_tensor = y_test_tensor.squeeze(-1)

# Initialize model
model = StockPredictorLSTM()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 100
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    
    # Backward pass and optimize
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

/home/harry/Documents/Code/Data_Science/DeepLearning/.venv_tf/lib/python3.12/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([5038, 1])) that is different to the input size (torch.Size([5038, 7])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [10/100], Loss: 0.0425
Epoch [20/100], Loss: 0.0185
Epoch [30/100], Loss: 0.0159
Epoch [40/100], Loss: 0.0149
Epoch [50/100], Loss: 0.0122
Epoch [60/100], Loss: 0.0072
Epoch [70/100], Loss: 0.0025
Epoch [80/100], Loss: 0.0021
Epoch [90/100], Loss: 0.0013
Epoch [100/100], Loss: 0.0008


In [62]:
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    test_loss = criterion(test_outputs, y_test_tensor)
    print(f'Test Loss: {test_loss.item():.4f}')
    
    # Inverse transform to get actual prices for all 7 days
    predicted_prices = scaler.inverse_transform(test_outputs.numpy().reshape(-1, 1)).reshape(test_outputs.shape)
    actual_prices = scaler.inverse_transform(y_test_tensor.numpy().reshape(-1, 1)).reshape(y_test_tensor.shape)

Test Loss: 0.0113


/home/harry/Documents/Code/Data_Science/DeepLearning/.venv_tf/lib/python3.12/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([1260, 1])) that is different to the input size (torch.Size([1260, 7])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [63]:
def predict_next_seven_days(model, last_sequence, scaler):
    model.eval()
    with torch.no_grad():
        last_sequence_tensor = torch.FloatTensor(last_sequence)
        prediction = model(last_sequence_tensor)  # shape: (1, 7)
        predicted_prices = scaler.inverse_transform(prediction.numpy().reshape(-1, 1)).flatten()
    return predicted_prices

# Get the last sequence from your data
last_sequence = scaled_prices[-SEQ_LENGTH:].reshape(1, SEQ_LENGTH, 1)
next_seven_days_prediction = predict_next_seven_days(model, last_sequence, scaler)
print("Predicted prices for next 7 days:", next_seven_days_prediction)

Predicted prices for next 7 days: [2314.4744 2532.5593 2131.2893 2001.5463 2275.495  2353.185  2267.914 ]
